# Prepare Dataset

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import os
os.chdir('/content/drive/MyDrive/AI_Advanced_CK/NLP/2_IMDB_reviews') # Chỉ định nơi làm việc (terminal)

In [31]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

# imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised = True)

In [32]:
# import numpy as np
# # train_data, test_data = imdb['train'], imdb['test']
# train_sentences = []
# test_sentences = []
# train_labels = []
# test_labels = []

In [33]:
import pandas as pd

# Đường dẫn tới file CSV (thay đổi theo đường dẫn thực tế của bạn)
train_csv_path = './train.csv'
test_csv_path = './test.csv'

# Đọc dữ liệu từ file CSV
train_data = pd.read_csv(train_csv_path)
test_data = pd.read_csv(test_csv_path)

# Khởi tạo các danh sách để lưu dữ liệu
train_sentences = []
test_sentences = []
train_labels = []
test_labels = []

# Lưu dữ liệu từ train.csv vào các danh sách
for index, row in train_data.iterrows():
    train_sentences.append(row['Title'] + " " + row['Description'])
    train_labels.append(row['Class Index'])

# Lưu dữ liệu từ test.csv vào các danh sách
for index, row in test_data.iterrows():
    test_sentences.append(row['Title'] + " " + row['Description'])
    test_labels.append(row['Class Index'])

# Hiển thị kết quả để kiểm tra
print(f'Train sentences: {train_sentences[:3]}')  # Hiển thị 3 câu đầu tiên để kiểm tra
print(f'Train labels: {train_labels[:3]}')        # Hiển thị 3 nhãn đầu tiên để kiểm tra
print(f'Test sentences: {test_sentences[:3]}')    # Hiển thị 3 câu đầu tiên để kiểm tra
print(f'Test labels: {test_labels[:3]}')          # Hiển thị 3 nhãn đầu tiên để kiểm tra

KeyboardInterrupt: 

In [ ]:
# first_element = next(iter(train_data.take(1)))

In [ ]:
for s,l in train_data:
  train_sentences.append(str(s.numpy()))
  train_labels.append(l.numpy())
for s,l in test_data:
  test_sentences.append(str(s.numpy()))
  test_labels.append(l.numpy())

In [ ]:
print(len(train_sentences))
print(len(train_labels))

## Add Vocab

In [ ]:
# s = 'Nguyen Van Manh'
s = 'Nguyễn Văn Mạnh'
train_sentences.append(s)
l = 1
train_labels = np.append(train_labels, l)

print(train_sentences[-1])
print(train_labels[-1])

In [ ]:
print(len(train_sentences))
print(len(train_labels))
print(train_sentences[0])
print(train_labels[0])
print(train_labels) # label chỉ có 2 nhãn : 0 và 1

In [ ]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Const - Variable

In [ ]:
vocab_size = 10000
embedding = 64
embedding_dim = 128  # Kích thước vector nhúng
max_length = 140 # 140 , 256
units = 128

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer (num_words=vocab_size, oov_token="<OOV>")
# tokenizer.fit_on_texts (train_sentences + test_sentences)
tokenizer.fit_on_texts (train_sentences + test_sentences + ['Nguyễn Văn Mạnh']) # Cách 2

In [ ]:
train_sentences = tokenizer.texts_to_sequences(train_sentences)
padded_train_sentences = pad_sequences (train_sentences, maxlen=max_length, truncating='post', padding='post')

In [ ]:
word_index = tokenizer.word_index

In [ ]:
print(word_index['romance'])
print(word_index['nguyen'])
print(word_index['van'])
# đầu vào là 'Nguyễn Văn Mạnh'
print(word_index['nguyễn'])
print(word_index['văn'])
print(word_index['mạnh']) # sẽ chuyển thành in thường hết

In [ ]:
# Từ index lấy ra word
reverse_word_index = {index: word for word, index in word_index.items()}
print(reverse_word_index[890])
print(reverse_word_index[24111])
print(reverse_word_index[1186])
print(reverse_word_index[86540])
print(reverse_word_index[86541])
print(reverse_word_index[86542])

In [ ]:
print(train_sentences[0])

In [ ]:
test_sentences = tokenizer.texts_to_sequences (test_sentences)
padded_test_sentences = pad_sequences(test_sentences, maxlen=max_length, truncating='post', padding='post')

In [ ]:
print(test_sentences[0])

In [ ]:
padded_train_sentences
# padded_train_sentences.shape

# Model

## RNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

rnn_model  = Sequential()
rnn_model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
rnn_model.add(SimpleRNN(units))
rnn_model.add(Dense(10, activation='relu'))
rnn_model.add(Dense(1, activation='sigmoid'))
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
rnn_model.summary()

- input_dim là output của lớp trước
- Embedding : Param = embedding*vocab_size = 64 * 10000 = 640000    
- SimpleRNN : Param = (units * units + units * input_dim + units) = (128 * 128 + 128 * 64 + 128) =  24704 (output của lớp trước = 64 chính là embending)
- Dense : Param = output_dim * input_dim + output_dim = 10 * 128 + 10 = 1290 (mấy lớp FC thì cũng tính giống như bên CNN)
- Dense : Param = output_dim * input_dim + output_dim = 1 * 10 + 1 = 11

- Total Param = 640000 + 24704 + 1290 + 11 = 666005

## LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

lstm_model = Sequential()
lstm_model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
lstm_model.add(LSTM(units))
lstm_model.add(Dense(10, activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
lstm_model.summary()

- input_dim là output của lớp trước
- Embedding : Param = embedding*vocab_size = 64 * 10000 = 640000    
- LSTM = RNN * 4 : Param = (units * units + units * input_dim + units) * 4 = (128 * 128 + 128 * 64 + 128) * 4  =  24704 * 4 = 98816 (output của lớp trước = 64 chính là embending)
- Dense : Param = output_dim * input_dim + output_dim = 10 * 128 + 10 = 1290 (mấy lớp FC thì cũng tính giống như bên CNN)
- Dense : Param = output_dim * input_dim + output_dim = 1 * 10 + 1 = 11

- Total Param = 640000 + 98816 + 1290 + 11 = 740117  

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Bidirectional, Dense, Flatten
# lstm_model2 = Sequential()
# lstm_model2.add(Embedding(vocab_size, embedding, input_length = max_length)) # embedding ở đây chính là output_dim
# lstm_model2.add(Flatten())
# # lstm_model2.add(LSTM(embedding)) # embedding node ẩn ???
# lstm_model2.add(Dense(10, activation='relu'))
# lstm_model2.add(Dense(1, activation='sigmoid'))
# lstm_model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# lstm_model2.summary()

## BiLSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

bilstm_model = Sequential()
bilstm_model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
bilstm_model.add(Bidirectional(LSTM(units)))
bilstm_model.add(Dense(10, activation='relu'))
bilstm_model.add(Dense(1, activation='sigmoid'))
bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
bilstm_model.summary()

( bidirectional (Bidirectional  (None, 256)  thì unit là 128))
- input_dim là output của lớp trước
- Embedding : Param = embedding*vocab_size = 64 * 10000 = 640000    
- BiLSTM = LSTM * 2 = RNN * 4 * 2  : Param = (units * units + units * input_dim + units) * 4 * 2 = (128 * 128 + 128 * 64 + 128) * 4 * 2  =  24704 * 4 = 197632 (output của lớp trước = 64 chính là embending)
- Dense : Param = output_dim * input_dim + output_dim = 10 * 256 + 10 = 2570 (mấy lớp FC thì cũng tính giống như bên CNN)
- Dense : Param = output_dim * input_dim + output_dim = 1 * 10 + 1 = 11

- Total Param = 640000 + 197632 + 2570 + 11 = 840213   

## Testing Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Dropout

new_model = Sequential()
new_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
new_model.add(LSTM(units=128, return_sequences=True))
new_model.add(Dropout(0.5))
new_model.add(LSTM(units=128))
new_model.add(Dropout(0.5))
new_model.add(Dense(10, activation='relu'))
new_model.add(Dense(1, activation='sigmoid'))
new_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
new_model.summary()

# Fit Model

In [ ]:
# model = rnn_model
# model = lstm_model
# model = lstm_model2
model = bilstm_model
# model = new_model

In [ ]:
# Callbacks
# patience = n // sau n eporch mà val_loss không được cải thiện thì dừng train
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=9, restore_best_weights=True)
checkpoint = ModelCheckpoint('./Manh_Save/best_model.h5', monitor='val_loss', verbose=1, save_best_only=True)

# Fit the model
H = bilstm_model.fit(padded_train_sentences, train_labels, epochs=10,
                            validation_data=(padded_test_sentences, test_labels),
                            callbacks=[early_stopping, checkpoint])

# Accessing training history
print("Training History:", H.history)

In [ ]:
train_acc = H.history['accuracy']
train_loss = H.history['loss']

val_acc = H.history['val_accuracy']
val_loss = H.history['val_loss']

index_loss = np.argmin(val_loss)
index_acc = np.argmax(val_acc)

val_lowest = val_loss[index_loss]
val_highest = val_acc[index_acc]

Epochs = [i+1 for i in range(len(train_acc))]

loss_label = f'Best Epoch = {str(index_loss + 1)}'
acc_label = f'Best Epoch = {str(index_acc + 1)}'

plt.figure(figsize= (20,8))
plt.style.use('fivethirtyeight')

plt.subplot(1,2,1)
plt.plot(Epochs , train_loss , 'r', label = 'Training Loss')
plt.plot(Epochs , val_loss , 'g' , label = 'Validation Loss')
plt.scatter(index_loss +1 , val_lowest , s = 150 , c = 'blue' , label = loss_label)
plt.title('Training vs Validation (loss)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(Epochs , train_acc , 'r', label= 'Training Accuracy')
plt.plot(Epochs , val_acc , 'g' , label = 'Validation Accuracy')
plt.scatter(index_acc + 1 , val_highest , s= 150 , c = 'blue' , label= acc_label)
plt.title('Training vs Validation (Accuracy)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

In [ ]:
# import numpy as np
# # Chuyển đổi dữ liệu nhãn thành mảng NumPy
# train_labels = np.array(train_labels)
# test_labels = np.array(test_labels)
# # Tiếp tục sử dụng phương thức fit
# model.fit(padded_test_sentences, train_labels, epochs=10, validation_data=(padded_test_sentences, test_labels))

# Test

## Write TSV

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
weights.shape # (10000, 64)

In [ ]:
import io
out_v = io.open('./Manh_Save/vecs.tsv', 'w', encoding = 'utf-8')
out_m = io.open('./Manh_Save/meta.tsv', 'w', encoding = 'utf-8')
for idx in range(1, vocab_size):
  word = tokenizer.index_word[idx]
  vector = weights[idx]
  out_m.write(word + '\n')
  out_v.write('\t'.join([str(x) for x in vector]) + '\n')
out_m.close()
out_v.close()

## Predict

In [ ]:
# Test the model with an additional sentence
additional_test_sen = ['this movie is amazing']
additional_test_seq = tokenizer.texts_to_sequences(additional_test_sen)
padded_additional_test_seq = pad_sequences(additional_test_seq, maxlen=max_length, truncating='post', padding='post')
print(padded_additional_test_seq)
print(padded_additional_test_seq.shape)

# Predict sentiment of the additional test sentence
additional_prediction = model.predict(padded_additional_test_seq)

# Round the sentiment prediction to get binary output
binary_prediction = 1 if additional_prediction >= 0.5 else 0
print("Binary sentiment prediction for 'this movie is amazing':", binary_prediction)

## Weight

In [ ]:
word = 'nguyễn'

if word in tokenizer.word_index:
    position = tokenizer.texts_to_sequences([word])
    weights = model.layers[0].get_weights()[0]
    print(weights[position])
else:
    print("Từ '{}' không có trong từ điển.".format(word))


## Embedings Từ

In [ ]:
word = 'nguyễn'

modelx = Sequential()
modelx.add(Embedding(vocab_size, embedding_dim, input_length=max_length))

if word in tokenizer.word_index:
    position = tokenizer.texts_to_sequences([word])
    print(f"Chỉ số của từ '{word}':", position[0][0])
    print("Chuỗi chứa chỉ số của từ:", position)
    embedding_of_word = modelx.predict(position)
    print(f"Embedding của từ '{word}':", embedding_of_word)

else:
    print("Từ '{}' không có trong từ điển.".format(word))

## Embedings Câu

In [ ]:
additional_test_sen = ["nguyễn văn mạnh là ai vậy ?"]
additional_test_seq = tokenizer.texts_to_sequences(additional_test_sen)
padded_additional_test_seq = pad_sequences(additional_test_seq, maxlen=max_length, truncating='post', padding='post')
print("Chuỗi chứa chỉ số của câu:", padded_additional_test_seq)

# Predict sentiment of the additional test sentence
modelx = Sequential()
modelx.add(Embedding(vocab_size, embedding_dim, input_length=max_length))

# Lấy embedding của chuỗi
embeddings = modelx.predict(padded_additional_test_seq)
print("Embeddings:", embeddings)